In [2]:
import rdflib
import networkx as nx
%matplotlib inline

INFO:rdflib:RDFLib Version: 4.2.1
INFO:root:Generating grammar tables from /usr/lib/python2.7/lib2to3/Grammar.txt
INFO:root:Generating grammar tables from /usr/lib/python2.7/lib2to3/PatternGrammar.txt


In [85]:
# Parse all triples from file and put them in NetworkX DiGraph
graph2 = nx.DiGraph()
g=rdflib.Graph()
g.load('lincoln-abox', format='turtle')
for s,p,o in g:      
    graph2.add_edge(s, o, name=p)

# For our nodes in our newly created DiGraph, we extract type assertions from dbpedia ontology
for node in graph2.nodes():
    g.load(node)
    for s,p,o in g:  
        if str(p) == 'http://www.w3.org/1999/02/22-rdf-syntax-ns#type' and 'dbpedia.org/ontology' in str(o):
            graph2.add_edge(s, o, name=p)

# Extract all paths with max. length = L
L = 2
paths = []
for node in graph2.nodes_iter():
    if graph2.out_degree(node):
        for path in (nx.all_simple_paths(graph2, node, target, L) for target in graph2.nodes_iter()):
            path = list(path)
            if path:
                path=list(path)[0]
                new_path = [path[0]]
                for i in range(len(path)-1):
                    new_path.append(graph2[path[i]][path[i+1]]['name'])
                    new_path.append(path[i+1])
                new_path.append(path[-1])
                print(new_path)
                print('-'*50)
                paths.append(new_path)

# Can we find one of our paths back in our graph?    
test_path = paths[1]
for test_path in paths:
    found = 1
    print(test_path)
    current_node = graph2
    for i in range(0, len(test_path), 2):
        if not test_path[i] in current_node:
            found = 0
            break
    print('found' if found else 'not found')

[rdflib.term.URIRef(u'http://dbpedia.org/resource/Abraham_Lincoln'), rdflib.term.URIRef(u'http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), rdflib.term.URIRef(u'http://dbpedia.org/ontology/OfficeHolder'), rdflib.term.URIRef(u'http://dbpedia.org/ontology/OfficeHolder')]
--------------------------------------------------
[rdflib.term.URIRef(u'http://dbpedia.org/resource/Abraham_Lincoln'), rdflib.term.URIRef(u'http://dbpedia.org/ontology/termPeriod'), rdflib.term.URIRef(u'http://dbpedia.org/resource/Abraham_Lincoln__1'), rdflib.term.URIRef(u'http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), rdflib.term.URIRef(u'http://dbpedia.org/ontology/TimePeriod'), rdflib.term.URIRef(u'http://dbpedia.org/ontology/TimePeriod')]
--------------------------------------------------
[rdflib.term.URIRef(u'http://dbpedia.org/resource/Abraham_Lincoln'), rdflib.term.URIRef(u'http://dbpedia.org/ontology/termPeriod'), rdflib.term.URIRef(u'http://dbpedia.org/resource/Abraham_Lincoln__1'), rdflib.term.URIRef(u'ht